In [137]:
pip install rake_nltk

Note: you may need to restart the kernel to use updated packages.


In [138]:
import warnings
warnings.filterwarnings('ignore')

In [139]:
import pandas as pd
import numpy as np
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity

In [140]:
help(Rake)

Help on class Rake in module rake_nltk.rake:

class Rake(builtins.object)
 |  Rapid Automatic Keyword Extraction Algorithm.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, stopwords=None, punctuations=None, language='english', ranking_metric=<Metric.DEGREE_TO_FREQUENCY_RATIO: 0>, max_length=100000, min_length=1)
 |      Constructor.
 |      
 |      :param stopwords: List of Words to be ignored for keyword extraction.
 |      :param punctuations: Punctuations to be ignored for keyword extraction.
 |      :param language: Language to be used for stopwords
 |      :param max_length: Maximum limit on the number of words in a phrase
 |                         (Inclusive. Defaults to 100000)
 |      :param min_length: Minimum limit on the number of words in a phrase
 |                         (Inclusive. Defaults to 1)
 |  
 |  extract_keywords_from_sentences(self, sentences)
 |      Method to extract keywords from the list of sentences provided.
 |      
 |      :param sentences: Te

In [141]:
help(cosine_similarity)

Help on function cosine_similarity in module sklearn.metrics.pairwise:

cosine_similarity(X, Y=None, dense_output=True)
    Compute cosine similarity between samples in X and Y.
    
    Cosine similarity, or the cosine kernel, computes similarity as the
    normalized dot product of X and Y:
    
        K(X, Y) = <X, Y> / (||X||*||Y||)
    
    On L2-normalized data, this function is equivalent to linear_kernel.
    
    Read more in the :ref:`User Guide <cosine_similarity>`.
    
    Parameters
    ----------
    X : ndarray or sparse array, shape: (n_samples_X, n_features)
        Input data.
    
    Y : ndarray or sparse array, shape: (n_samples_Y, n_features)
        Input data. If ``None``, the output will be the pairwise
        similarities between all samples in ``X``.
    
    dense_output : boolean (optional), default True
        Whether to return dense output even when the input is sparse. If
        ``False``, the output is sparse if both input arrays are sparse.
    
 

In [142]:
df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')

In [143]:
df = df[['Title','Genre','Director','Actors','Plot']]
df

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...
...,...,...,...,...,...
245,The Lost Weekend,"Drama, Film-Noir",Billy Wilder,"Ray Milland, Jane Wyman, Phillip Terry, Howard...",The desperate life of a chronic alcoholic is f...
246,Short Term 12,Drama,Destin Daniel Cretton,"Brie Larson, John Gallagher Jr., Stephanie Bea...",A 20-something supervising staff member of a r...
247,His Girl Friday,"Comedy, Drama, Romance",Howard Hawks,"Cary Grant, Rosalind Russell, Ralph Bellamy, G...",A newspaper editor uses every trick in the boo...
248,The Straight Story,"Biography, Drama",David Lynch,"Sissy Spacek, Jane Galloway Heitz, Joseph A. C...",An old man makes a long journey by lawn-mover ...


In [144]:
df['key_words'] = ''
for index,row in df.iterrows():
    plot = row['Plot']
    r = Rake()
    r.extract_keywords_from_text(plot)
    key_word_score_in_dict = r.get_word_degrees()
    row['key_words'] = list(key_word_score_in_dict)

In [145]:
df.drop(columns = 'Plot',inplace = True)

In [146]:
df['Actors'] = df['Actors'].apply(lambda x : x.split(',')[:3])

In [147]:
df['Genre'] = df['Genre'].apply(lambda x : x.split(','))

In [148]:
df['Director'] = df['Director'].apply(lambda x : x.split())

In [149]:
df['Director'] = df['Director'].apply(lambda x : ''.join(word.lower() for word in x))


In [150]:
df['Actors'] = df['Actors'].apply(lambda x : [word.lower().replace(' ','') for word in x])

In [151]:
df.set_index('Title',inplace = True)

In [152]:
df

,Genre,Director,Actors,key_words
Title,,,,
The Shawshank Redemption,"[Crime, Drama]",frankdarabont,"[timrobbins, morganfreeman, bobgunton]","[number, eventual, redemption, common, decency..."
The Godfather,"[Crime, Drama]",francisfordcoppola,"[marlonbrando, alpacino, jamescaan]","[organized, crime, dynasty, transfers, control..."
The Godfather: Part II,"[Crime, Drama]",francisfordcoppola,"[alpacino, robertduvall, dianekeaton]","[grip, tightens, family, crime, syndicate, car..."
The Dark Knight,"[Action, Crime, Drama]",christophernolan,"[christianbale, heathledger, aaroneckhart]","[people, chaos, gotham, physical, tests, abili..."
12 Angry Men,"[Crime, Drama]",sidneylumet,"[martinbalsam, johnfiedler, leej.cobb]","[jury, holdout, attempts, forcing, miscarriage..."
...,...,...,...,...
The Lost Weekend,"[Drama, Film-Noir]",billywilder,"[raymilland, janewyman, phillipterry]","[chronic, alcoholic, followed, four, desperate..."
Short Term 12,[Drama],destindanielcretton,"[brielarson, johngallagherjr., stephaniebeatriz]","[world, alongside, worker, something, supervis..."
His Girl Friday,"[Comedy, Drama, Romance]",howardhawks,"[carygrant, rosalindrussell, ralphbellamy]","[ace, reporter, ex, wife, newspaper, editor, u..."


In [153]:
df['bag_of_words'] = ' '
columns = df.columns
for index,row in df.iterrows():
    words = ''
    for col in columns:
        if col!= 'Director':
            words = words + ' '.join(row[col])
        else:
            words = words + row[col] + ' '
    row['bag_of_words'] = words

In [154]:
new_df = df[['bag_of_words']]
new_df

,bag_of_words
Title,
The Shawshank Redemption,Crime Dramafrankdarabont timrobbins morganfre...
The Godfather,Crime Dramafrancisfordcoppola marlonbrando al...
The Godfather: Part II,Crime Dramafrancisfordcoppola alpacino robert...
The Dark Knight,Action Crime Dramachristophernolan christian...
12 Angry Men,Crime Dramasidneylumet martinbalsam johnfiedl...
...,...
The Lost Weekend,Drama Film-Noirbillywilder raymilland janewym...
Short Term 12,Dramadestindanielcretton brielarson johngallag...
His Girl Friday,Comedy Drama Romancehowardhawks carygrant ro...


In [155]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
matrix = count.fit_transform(new_df['bag_of_words'])

In [156]:
indices = pd.Series(new_df.index)

In [157]:
cosine_sim = cosine_similarity(matrix,matrix)

In [158]:
cosine_sim

array([[1.        , 0.11764706, 0.10114435, ..., 0.        , 0.        ,
        0.        ],
       [0.11764706, 1.        , 0.35400522, ..., 0.        , 0.        ,
        0.        ],
       [0.10114435, 0.35400522, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [159]:
title = "The Lion King"
recommended_movies = []
idx = indices[indices == title].index[0]
score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
top_10_index = list(score_series.iloc[1:11].index)
for i in top_10_index:
    recommended_movies.append(list(df.index)[i])

In [160]:
movies_to_recommend_index

[52, 181, 145, 98, 234, 147, 243, 25, 151, 228]

In [161]:
recommended_movies

['Monsters, Inc.',
 'Song of the Sea',
 'Toy Story',
 'Monty Python and the Holy Grail',
 'The Nightmare Before Christmas',
 'WALL·E',
 'Finding Nemo',
 'Aladdin',
 'Zootopia',
 'Up']